# keyboard_config

> Segmentation-specific keyboard actions, modes, and zone configuration

In [ ]:
#| default_exp components.keyboard_config

In [ ]:
#| export
from typing import Tuple

# Keyboard navigation library
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone

# Card stack library
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.keyboard.actions import (
    create_card_stack_focus_zone, create_card_stack_nav_actions,
)

# Token selector library
from cjm_fasthtml_token_selector.keyboard.actions import (
    create_token_selector_mode, create_token_nav_actions,
)
from cjm_transcript_segmentation.components.card_stack_config import (
    SEG_TS_CONFIG,
)

## Hidden Input and Button IDs

Constants for hidden inputs and HTMX trigger buttons used by the keyboard navigation system.

In [ ]:
#| export
# Button IDs for keyboard actions — segmentation operations (workflow-specific)
SD_SEG_ENTER_SPLIT_BTN = "sd-seg-enter-split-btn"
SD_SEG_EXIT_SPLIT_BTN = "sd-seg-exit-split-btn"
SD_SEG_SPLIT_BTN = "sd-seg-split-btn"
SD_SEG_MERGE_BTN = "sd-seg-merge-btn"
SD_SEG_UNDO_BTN = "sd-seg-undo-btn"

## Segmentation Keyboard Parts

Returns the segmentation-specific building blocks (zone, actions, modes) for assembly
into a shared `ZoneManager` by the combined-level keyboard config.

In [ ]:
#| export
def create_seg_kb_parts(
    ids:CardStackHtmlIds,  # Card stack HTML IDs
    button_ids:CardStackButtonIds,  # Card stack button IDs for navigation
    config:CardStackConfig,  # Card stack configuration
) -> Tuple[FocusZone, tuple, tuple]:  # (zone, actions, modes)
    """Create segmentation-specific keyboard building blocks.

    Returns a zone, actions tuple, and modes tuple for assembly into a shared
    ZoneManager by the combined-level keyboard config.
    """
    # Card stack zone from library
    card_zone = create_card_stack_focus_zone(
        ids=ids,
        on_focus_change="onCardFocusChange",
        data_attributes=("segment-index",),
    )
    zone_id = card_zone.id

    # Card stack navigation actions from library (arrows, page jump, first/last, width)
    nav_actions = create_card_stack_nav_actions(
        zone_id=zone_id,
        button_ids=button_ids,
        config=config,
        disable_in_modes=("split",),
    )

    # --- Token selector: split mode definition (library factory) ---
    split_mode = create_token_selector_mode(
        SEG_TS_CONFIG,
        mode_name="split",
        indicator_text="Split Mode",
        exit_key="",
        exit_on_zone_change=False,
    )

    # --- Token selector: Home/End navigation in split mode ---
    token_nav_actions = create_token_nav_actions(
        SEG_TS_CONFIG,
        zone_id=zone_id,
        mode_name="split",
    )

    # --- Consumer-specific: segmentation operation actions ---
    # All seg actions are zone-restricted to prevent firing when alignment zone is active
    seg_zone_ids = (zone_id,)

    seg_actions = (
        # Enter split mode (Enter or Space when NOT in split mode)
        KeyAction(
            key="Enter",
            htmx_trigger=SD_SEG_ENTER_SPLIT_BTN,
            zone_ids=seg_zone_ids,
            mode_enter="split",
            not_modes=("split",),
            description="Enter split mode",
            hint_group="Segmentation",
        ),
        KeyAction(
            key=" ",
            htmx_trigger=SD_SEG_ENTER_SPLIT_BTN,
            zone_ids=seg_zone_ids,
            mode_enter="split",
            not_modes=("split",),
            description="Enter split mode",
            hint_group="Segmentation",
            show_in_hints=False,
        ),

        # Execute split (Enter or Space when IN split mode)
        KeyAction(
            key="Enter",
            htmx_trigger=SD_SEG_SPLIT_BTN,
            zone_ids=seg_zone_ids,
            mode_exit=True,
            mode_names=("split",),
            description="Split at caret",
            hint_group="Split Mode",
        ),
        KeyAction(
            key=" ",
            htmx_trigger=SD_SEG_SPLIT_BTN,
            zone_ids=seg_zone_ids,
            mode_exit=True,
            mode_names=("split",),
            description="Split at caret",
            hint_group="Split Mode",
            show_in_hints=False,
        ),

        # Exit split mode (Escape when IN split mode)
        KeyAction(
            key="Escape",
            htmx_trigger=SD_SEG_EXIT_SPLIT_BTN,
            zone_ids=seg_zone_ids,
            mode_exit=True,
            mode_names=("split",),
            description="Exit split mode",
            hint_group="Split Mode",
        ),

        # Merge with previous (Backspace when NOT in split mode)
        KeyAction(
            key="Backspace",
            htmx_trigger=SD_SEG_MERGE_BTN,
            zone_ids=seg_zone_ids,
            not_modes=("split",),
            description="Merge with previous",
            hint_group="Segmentation",
        ),

        # Undo (Ctrl+Z in any mode, but zone-restricted)
        KeyAction(
            key="z",
            modifiers=frozenset({"ctrl"}),
            htmx_trigger=SD_SEG_UNDO_BTN,
            zone_ids=seg_zone_ids,
            description="Undo",
            hint_group="General",
        ),
    )

    actions = (*nav_actions, *seg_actions, *token_nav_actions)
    modes = (split_mode,)

    return card_zone, actions, modes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()